# Libraries

In [1]:
import numpy as np
from scipy import stats #scipy is the library; stats is the module within the library
from scipy.stats import norm, expon, binom, poisson
import math
import pandas as pd

#images
from IPython.display import Image

#visualization
import seaborn as sns
sns.set_style('darkgrid')

from matplotlib import pyplot as plt #matplotlib is the library, and pyplot is the module
%matplotlib inline

----

# 2.21- Covariance and Correlation

### COVARIANCE

- correlation of -1  means a perfect inverse correlation
- correlation of 0: no correlation
- correlation 1: perfect correlation

**IMPORTANT**: Correlation does not imply causation
- Only controlled, randomized experiment can give insights into causation
- Use correlation to decide what experiments to conduct